**Importation des bibliothèques**


In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import (
    accuracy_score, f1_score, hamming_loss
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from tqdm import tqdm

**Classe pour charger les données**

In [ ]:
class DataLoader:
    def load_data(file_path: str):
        """Charge et prépare les données pour la classification multi-label"""
        file_path = "books_dataset.xls"
        df = pd.read_csv(file_path)

        # Nettoyage des colonnes
        df = df.dropna(subset=['description', 'categories'])
        df['description'] = df['description'].fillna('').str.strip()
        df['categories'] = df['categories'].astype(str).str.split(', ')

        # Conversion des catégories en matrice binaire
        mlb = MultiLabelBinarizer()
        y = mlb.fit_transform(df['categories'])

        return df['description'], y, mlb

**Classe pour prétraiter les donnée**


In [ ]:
class Preprocessor(TransformerMixin, BaseEstimator):
    """Préprocesseur pour nettoyer les descriptions et appliquer TF-IDF"""
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            stop_words='english', max_features=5000, min_df=2, max_df=0.95
        )

    def fit(self, X, y=None):
        X = pd.Series(X).astype(str)
        self.vectorizer.fit(X)
        return self

    def transform(self, X):
        X = pd.Series(X).astype(str)
        return self.vectorizer.transform(X)


**Classe pour entraîner et évaluer les modèles**


In [ ]:
class ModelTrainer:
    def __init__(self, models, metrics):
        self.models = models
        self.metrics = metrics
        self.results = []

    def train_and_evaluate(self, X_train, X_test, y_train, y_test):
        """Entraîne et évalue tous les modèles"""
        for model_name, model in tqdm(self.models.items(), desc="Training models"):
            model.fit(X_train, y_train)  # Entraîner le modèle
            predictions = model.predict(X_test)  # Prédire sur les données de test

            # Calculer toutes les métriques
            metrics_results = {
                metric_name: metric_func(y_test, predictions)
                for metric_name, metric_func in self.metrics.items()
            }

            # Sauvegarder les résultats
            self.results.append({'model': model, 'name': model_name, **metrics_results})



**Classe pour exécuter l'entraînement**


In [ ]:
import os
import pickle
from sklearn.model_selection import train_test_split

class ExperimentRunner:
    def __init__(self, config):
        self.config = config

    def run(self, X, y):
        """Exécute l'expérience complète."""
        # Diviser les données en entraînement et test
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.config['test_size'], random_state=42
        )

        # Filtrer les colonnes avec des classes vides dans l'ensemble d'entraînement
        valid_columns = y_train.sum(axis=0) > 0
        y_train = y_train[:, valid_columns]
        y_test = y_test[:, valid_columns]

        # Prétraiter les données
        preprocessor = self.config['preprocessor']()
        X_train_transformed = preprocessor.fit_transform(X_train)
        X_test_transformed = preprocessor.transform(X_test)

        # Entraîner et évaluer les modèles
        trainer = ModelTrainer(self.config['models'], self.config['metrics'])
        trainer.train_and_evaluate(X_train_transformed, X_test_transformed, y_train, y_test)

        # Sauvegarder les modèles avec pickle
        self.save_models(trainer.results, "saved_models")

        return trainer.results

    def save_models(self, results, save_dir):
        """Sauvegarde les modèles entraînés avec pickle."""
        os.makedirs(save_dir, exist_ok=True)

        for result in results:
            model_obj = result['model']  # Objet du modèle entraîné
            model_name = result['name'].replace(' ', '_').lower()  # Nom du modèle

            # Vérification avant sauvegarde
            if not hasattr(model_obj, "fit") or not hasattr(model_obj, "predict"):
                print(f"Le modèle {model_name} n'est pas valide. Ignoré.")
                continue

            # Sauvegarde du modèle
            file_path = os.path.join(save_dir, f"{model_name}.pkl")
            with open(file_path, "wb") as file:
                pickle.dump(model_obj, file)
                print(f"Modèle sauvegardé dans {file_path}")

    def load_model(self, filepath):
        """Charge un modèle entraîné depuis un fichier pickle."""
        with open(filepath, "rb") as file:
            model = pickle.load(file)
            print(f"Modèle chargé depuis {filepath}")
            return model


**Configuration de l'expérience**


In [ ]:
CONFIG = {
    'test_size': 0.2,  # Pourcentage des données pour le test
    'preprocessor': Preprocessor,  # Classe de prétraitement
    'models': {  # Modèles à entraîner
        'Random Forest': OneVsRestClassifier(RandomForestClassifier(n_estimators=10, random_state=42)),
        'k-Nearest Neighbors': OneVsRestClassifier(KNeighborsClassifier(n_neighbors=3)),
        'Gradient Boosting': OneVsRestClassifier(GradientBoostingClassifier(n_estimators=10, random_state=42))
    },
    'metrics': {  # Métriques pour l'évaluation
        'hamming_loss': hamming_loss,
        'subset_accuracy': accuracy_score,
        'micro_f1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='micro', zero_division=1),
        'macro_f1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro', zero_division=1)
    }
}

**Exécution principal**

In [ ]:
if __name__ == "__main__":
    # Charger les données
    file_path = "books_dataset.xls"

    # Charger et préparer les données
    X, y, mlb = DataLoader.load_data(file_path)
    CONFIG['mlb'] = mlb

    # Lancer l'expérience
    runner = ExperimentRunner(CONFIG)
    results = runner.run(X, y)

    # Afficher les résultats
    results_df = pd.DataFrame(results)
    print("Résultats de l'entraînement :")
    print(results_df)

    # Vérification de la sauvegarde
    print("Les modèles ont été sauvegardés dans le répertoire 'saved_models'.")

Training models: 100%|██████████| 3/3 [02:22<00:00, 47.62s/it]


Modèle sauvegardé dans saved_models/random_forest.pkl
Modèle sauvegardé dans saved_models/k-nearest_neighbors.pkl
Modèle sauvegardé dans saved_models/gradient_boosting.pkl
Résultats de l'entraînement :
                                               model                 name  \
0  OneVsRestClassifier(estimator=RandomForestClas...        Random Forest   
1  OneVsRestClassifier(estimator=KNeighborsClassi...  k-Nearest Neighbors   
2  OneVsRestClassifier(estimator=GradientBoosting...    Gradient Boosting   

   hamming_loss  subset_accuracy  micro_f1  macro_f1  
0      0.000570         0.865522  0.911776  0.901344  
1      0.001285         0.623748  0.767773  0.801033  
2      0.002046         0.574392  0.736000  0.870639  
Les modèles ont été sauvegardés dans le répertoire 'saved_models'.
